# ELoFTR demo resultados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/zju3dv/EfficientLoFTR.git
%cd EfficientLoFTR
!pip install opencv-python torch torchvision yacs matplotlib scipy tqdm --quiet

In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.spatial.distance import cdist
from scipy.stats import entropy

# Ajusta el path
from omegaconf import OmegaConf
from src.loftr import LoFTR

config = OmegaConf.load("configs/efficientloftr/loftr_realistic.ini")
matcher = LoFTR(config).eval().to("cpu")

def extract_eff_loftr(img1, img2):
    g1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)[None][None]
    g2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)[None][None]
    t1 = torch.from_numpy(g1 / 255.).float().to("cpu")
    t2 = torch.from_numpy(g2 / 255.).float().to("cpu")
    with torch.no_grad():
        data = {"image0": t1, "image1": t2}
        matches = matcher(data)
    kp1 = matches["keypoints0"].cpu().numpy()
    kp2 = matches["keypoints1"].cpu().numpy()
    desc1 = matches["descriptors0"].cpu().numpy()
    desc2 = matches["descriptors1"].cpu().numpy()
    return kp1, desc1, kp2, desc2

def spatial_entropy(kps, shape, grid_size=4):
    H, W = shape[:2]
    heatmap = np.zeros((grid_size, grid_size))
    for x, y in kps:
        c = min(int(x / W * grid_size), grid_size - 1)
        r = min(int(y / H * grid_size), grid_size - 1)
        heatmap[r, c] += 1
    p = heatmap.flatten()
    p /= (p.sum() + 1e-8)
    return entropy(p)

def match_metrics_draw(kp1, desc1, kp2, desc2, img1, img2, save_path):
    d = cdist(desc1, desc2, metric='euclidean')
    idx = np.argmin(d, axis=1)
    num = len(idx)
    eff = num / (len(desc1) + 1e-8)
    avgd = float(d[np.arange(num), idx].mean())
    se = spatial_entropy(kp1[idx], img1.shape)

    # draw
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    out = np.zeros((max(h1,h2), w1 + w2, 3), dtype=np.uint8)
    out[:h1, :w1] = img1
    out[:h2, w1:] = img2
    for i, j in enumerate(idx):
        p1 = tuple(np.round(kp1[i]).astype(int))
        p2 = tuple(np.round(kp2[j]).astype(int) + np.array([w1,0]))
        cv2.line(out, p1, p2, (0,255,0), 1)
    cv2.imwrite(save_path, out)

    return num, eff, avgd, se


In [ ]:
# 3. Iterar sobre pares day / night
BASE = '/content/drive/MyDrive/pruebas-pfc-2025/aachen_dataset/day'
OUT = '/content/efficientloftr_day'
os.makedirs(OUT, exist_ok=True)
results = []

for pair in tqdm(sorted(os.listdir(BASE))):
    p = os.path.join(BASE, pair)
    i1 = os.path.join(p, 'img1.png')
    i2 = os.path.join(p, 'img2.png')
    if os.path.exists(i1) and os.path.exists(i2):
        img1 = cv2.imread(i1)
        img2 = cv2.imread(i2)
        kp1, desc1, kp2, desc2 = extract_eff_loftr(img1, img2)
        num, eff, avgd, se = match_metrics_draw(kp1, desc1, kp2, desc2, img1, img2, os.path.join(OUT, f"{pair}_m.png"))
        results.append({
            'pair': pair,
            'num_matches': num,
            'efficiency': round(eff, 4),
            'avg_distance': round(avgd, 4),
            'spatial_entropy': round(se, 4)
        })

df = pd.DataFrame(results)
df.to_csv(os.path.join(OUT, 'efficientloftr_day.csv'), index=False)
print("Resultados guardados en:", OUT)